## Lens injection in multiple DC2 cutout images

In this notebook, we will show how one can inject simulated lenses to multiple DC2 cutout images and create a catalog of strong lense.

For this, we should follow following steps:
1. Create a lens population using sim-pipeline
2. Generate random ra and dec using random_ra_dec function in RSP module
3. Then, finally one can should call multiple_lens_injection function in RSP module

In [1]:
import sys
from astropy.cosmology import FlatLambdaCDM
from astropy.units import Quantity
from sim_pipeline.gg_lens_pop import GGLensPop
import lsst.daf.butler as dafButler
from sim_pipeline import rsp
from sim_pipeline.Util import param_util

In [ ]:
sys.path.insert(0, '/home/nkhadka/notebooks/mytutorials/sim-pipeline/')

## Generate Galaxy-Galaxy Lens Population

In [2]:
# define a cosmology
cosmo = FlatLambdaCDM(H0=70, Om0=0.3)

# define a sky area
sky_area = Quantity(value=0.1, unit='deg2')

# define limits in the intrinsic deflector and source population (in addition to the skypy config file)
kwargs_deflector_cut = {'band': 'g', 'band_max':28, 'z_min': 0.01, 'z_max': 2.5}
kwargs_source_cut = {'band': 'g', 'band_max':28, 'z_min': 0.1, 'z_max': 5.}

# run skypy pipeline and make galaxy-galaxy population class using GGLensPop
gg_lens_pop = GGLensPop(lens_type='all-galaxies', source_type='galaxies', 
                        kwargs_deflector_cut=kwargs_deflector_cut, kwargs_source_cut=kwargs_source_cut,
                        kwargs_mass2light=None, skypy_config=None, sky_area=sky_area, cosmo=cosmo)

## Initiate Butler
Butler is necessary to access and process DC2 data. So, one needs to initiate it. To initiate butler, one needs to specify data configuration and data collection.

In [3]:
config = "dp02"
collection ='2.2i/runs/DP0.2'
butler = dafButler.Butler(config, collections=collection)
skymap = butler.get("skyMap")

## Import rsp module
This module can be used to do all the operation necessary for the lens injection in DC2 data.
We will choose 100 random ra, dec pair in the DC2 data and we inject random gg lenses in these ra, dec position. Injected_lens_catalog contains following images and information,
1. Lens to be injected in r-band.
2. Corresponding DC2 cutout image in r-band.
3. r, g, and i band DC2 cutout images with corresponding injected lenes.
4. Center of the DC2 cutout images where lenses have been injected.

Rsp module is imported at the top

In [5]:
##provide minimum and maximum limits for ra and dec. Also, specify how many ra, dec pair you want.
ra, dec=param_util.random_ra_dec(55, 70, -43, -30, 100)

In [6]:
## specify pixel number and pixel scale of the images. Here we have chosen 64 and 0.2 respectively. 
## We should choose pixel scale = 0.2 to match the pixel scale scale of DC2 data.
## For this function one lens_cut. If not provided, default option is None.
lens_cut = {'min_image_separation': 0.8, 'max_image_separation': 10, 
                        'mag_arc_limit': {'g': 23, 'r': 23, 'i': 23}}
injected_lens_catalog=rsp.multiple_lens_injection(gg_lens_pop, ra, dec, 64, 0.2, butler, lens_cut=lens_cut, flux=None)

In [7]:
injected_lens_catalog

lens,cutout_image,injected_lens_r,injected_lens_g,injected_lens_i,cutout_center
"float32[64,64]","float32[64,64]","float32[64,64]","float32[64,64]","float32[64,64]",float64[2]
0.0 .. 0.0,-0.031025013 .. 0.04776547,-0.031025013 .. 0.04776547,0.035453714 .. 0.05214813,0.051779103 .. -0.029229892,60.85611332568641 .. -35.96722458736316
0.0 .. 0.0,0.012552995 .. 0.009242649,0.012552995 .. 0.009242649,-0.003614279 .. -0.020279886,-0.024247432 .. 0.04610421,65.40440769132046 .. -41.43389549336795
0.0 .. 0.0,0.0646337 .. -0.018245338,0.0646337 .. -0.018245338,-0.029428983 .. 0.0143132415,0.030992271 .. 0.0021171963,58.889121610697195 .. -32.4264822278856
0.0 .. 0.0,-0.10209689 .. 0.010986028,-0.10209689 .. 0.010986028,-0.005986939 .. -0.037921615,-0.04150873 .. -0.044987265,58.148975556561105 .. -37.337433140014745
0.0 .. 0.0,0.35131955 .. -0.020105552,0.35131955 .. -0.020105552,0.071492925 .. -0.0033082063,0.5467828 .. -0.011295935,60.53512969786194 .. -33.207681057624804
0.0 .. 0.0,0.29451466 .. 0.039429452,0.29451466 .. 0.039429452,0.16354737 .. 0.00405959,0.24544688 .. 0.023733584,58.83722441902085 .. -36.84964800487269
0.0 .. 0.0,-0.01410901 .. 0.064772144,-0.01410901 .. 0.064772144,-0.07242727 .. -0.0056932885,0.13690712 .. -0.047720823,68.53634984929118 .. -34.164567154229985
0.0 .. 0.0,0.05972465 .. -0.0023960597,0.05972465 .. -0.0023960597,0.023833506 .. 0.031867053,0.026590014 .. -0.10990596,59.41785466336982 .. -37.602660880508836
0.0 .. 0.0,0.017555153 .. -0.015944025,0.017555153 .. -0.015944025,-0.007915744 .. 0.008529122,-0.08528645 .. 0.036231413,58.396101890352185 .. -40.25136844953081
